In [1]:
from rbpnet import io

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = io.dataset_ops.load_tfrecord('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord', deserialize=False)
features = io.dataset_ops.features_from_json_file('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord.features.json')

In [3]:
features.deserialize_example(proto)

NameError: name 'proto' is not defined

In [58]:
torch.gather(torch.rand(8, 3, 4), dim=0, index=torch.tensor([0, 1, 7]))

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [61]:
torch.rand(5, 3, 4)[torch.tensor([True, False, True, False, False]), :, :].shape

torch.Size([2, 3, 4])

In [3]:
proto_samples = [proto for proto in dataset.as_numpy_iterator()]

In [4]:
len(proto_samples)

1147

In [4]:
import torch
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader

class TFRecordDataset(Dataset):
    def __init__(self, filepath, features_filepath=None):
        self._tf_dataset = io.dataset_ops.load_tfrecord(filepath, deserialize=False)
        self._serialized_protos = [proto for proto in self._tf_dataset.as_numpy_iterator()]

        # deserialize
        if features_filepath is None:
            features_filepath = filepath + '.features.json'
        self.features = io.dataset_ops.features_from_json_file(features_filepath)

    def __len__(self):
        return len(self._serialized_protos)

    def __getitem__(self, idx):
        example = self.features.deserialize_example(self._serialized_protos[idx])
        example = (example['inputs'], example['outputs'])
        example = tf.nest.map_structure(lambda x: torch.tensor(x.numpy()).to(torch.float32), example)
        return example

In [32]:
dataset = TFRecordDataset('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord')
dataset

In [33]:
example = dataset[2]
example

({'input': tensor([[0., 0., 0., 1.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.],
          ...,
          [0., 0., 1., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.]])},
 {'counts': {'control': tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
           1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 0., 0.,
           2., 0., 1., 1., 0., 4., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 2., 0.,
           1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
           0., 0., 0., 0., 0., 0., 3., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
           0., 0., 1., 0., 0., 2., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
           1., 0., 2., 0., 1., 0., 0., 2., 0., 0., 2., 0., 0., 2., 0., 0., 0., 1.,
           0., 0., 0., 1., 0., 1., 0.

In [44]:
dataloader = DataLoader(dataset, batch_size=128, num_workers=8)

In [52]:
for batch in dataloader:
    _ = batch

In [51]:
from torchrbpnet.data import tfrecord_to_dataloader

for batch in tfrecord_to_dataloader('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord'):
    _ = batch

In [63]:
mask_HepG2 = list()
with open('../example/experiments.txt') as f:
    for line in f:
        symbol, cell = line.strip().split('_')

        indicator = False
        if cell == 'HepG2':
            indicator = True
        mask_HepG2.append(indicator)
mask_HepG2 = torch.tensor(mask_HepG2)
torch.save(mask_HepG2, 'experiment-mask.HepG2.pt')

In [31]:
mask_K562 = list()
with open('../example/experiments.txt') as f:
    for line in f:
        symbol, cell = line.strip().split('_')

        indicator = False
        if cell == 'K562':
            indicator = True
        mask_K562.append(indicator)
mask_K562 = torch.tensor(mask_K562)
torch.save(mask_K562, 'experiment-mask.K562.pt')

In [32]:
mask_unique_ENCODE_prefer_HepG2 = list()
visited = set()
with open('../example/experiments.txt') as f:
    for line in f:
        symbol, cell = line.strip().split('_')

        indicator = False
        if symbol not in visited:
            indicator = True
            visited.add(symbol)
        print(symbol, indicator)
        mask_unique_ENCODE_prefer_HepG2.append(indicator)

mask_unique_ENCODE_prefer_HepG2 = torch.tensor(mask_unique_ENCODE_prefer_HepG2)
print(torch.sum(mask_unique_ENCODE_prefer_HepG2))
torch.save(mask_unique_ENCODE_prefer_HepG2, 'experiment-mask.ENCODE-150.prioritize-HepG2.pt')

AARS True
AATF True
ABCF1 True
AGGF1 True
AGGF1 False
AKAP1 True
AKAP1 False
AKAP8L True
APOBEC3C True
AQR True
AQR False
BCCIP True
BCLAF1 True
BUD13 True
BUD13 False
CDC40 True
CPEB4 True
CPSF6 True
CSTF2 True
CSTF2T True
CSTF2T False
DDX21 True
DDX24 True
DDX3X True
DDX3X False
DDX42 True
DDX51 True
DDX52 True
DDX52 False
DDX55 True
DDX55 False
DDX59 True
DDX6 True
DDX6 False
DGCR8 True
DGCR8 False
DHX30 True
DHX30 False
DKC1 True
DROSHA True
DROSHA False
EFTUD2 True
EFTUD2 False
EIF3D True
EIF3G True
EIF3H True
EIF4G2 True
EWSR1 True
EXOSC5 True
EXOSC5 False
FAM120A True
FAM120A False
FASTKD2 True
FASTKD2 False
FKBP4 True
FMR1 True
FTO True
FTO False
FUBP3 True
FUS True
FUS False
FXR1 True
FXR2 True
FXR2 False
G3BP1 True
GEMIN5 True
GNL3 True
GPKOW True
GRSF1 True
GRWD1 True
GRWD1 False
GTF2F1 True
GTF2F1 False
HLTF True
HLTF False
HNRNPA1 True
HNRNPA1 False
HNRNPC True
HNRNPC False
HNRNPK True
HNRNPK False
HNRNPL True
HNRNPL False
HNRNPM True
HNRNPM False
HNRNPU True
HNRNPU False


In [41]:
mask = mask_unique_ENCODE_prefer_HepG2
mask

tensor([ True,  True,  True,  True, False,  True, False,  True,  True,  True,
        False,  True,  True,  True, False,  True,  True,  True,  True,  True,
        False,  True,  True,  True, False,  True,  True,  True, False,  True,
        False,  True,  True, False,  True, False,  True, False,  True,  True,
        False,  True, False,  True,  True,  True,  True,  True,  True, False,
         True, False,  True, False,  True,  True,  True, False,  True,  True,
        False,  True,  True, False,  True,  True,  True,  True,  True,  True,
        False,  True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,  True,
        False,  True,  True,  True, False,  True,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,  True,
         True, False,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True, False,  True,  True,  True, 

In [51]:
mask.shape

torch.Size([223])

In [54]:
indices = torch.arange(0, 223)
print(indices.shape)
print(indices.dtype)

torch.Size([223])
torch.int64


In [60]:
selected_indices = torch.masked_select(indices, mask)
selected_indices

tensor([  0,   1,   2,   3,   5,   7,   8,   9,  11,  12,  13,  15,  16,  17,
         18,  19,  21,  22,  23,  25,  26,  27,  29,  31,  32,  34,  36,  38,
         39,  41,  43,  44,  45,  46,  47,  48,  50,  52,  54,  55,  56,  58,
         59,  61,  62,  64,  65,  66,  67,  68,  69,  71,  73,  75,  77,  79,
         81,  83,  85,  87,  89,  91,  92,  93,  95,  96,  98, 100, 102, 104,
        106, 108, 109, 110, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122,
        123, 125, 126, 127, 128, 129, 130, 131, 133, 135, 136, 137, 138, 140,
        142, 144, 146, 147, 148, 150, 151, 153, 154, 156, 157, 158, 159, 161,
        162, 163, 165, 167, 169, 171, 173, 174, 176, 177, 178, 179, 181, 183,
        184, 186, 188, 189, 191, 193, 195, 197, 199, 201, 203, 204, 205, 207,
        208, 209, 211, 213, 215, 216, 218, 219, 220, 222])

tensor([ 95, 119, 137,   8, 129,  31, 110, 144, 143,  84,  71,  65,  90, 135,
        121, 112, 147, 140, 130, 114], dtype=torch.int32)

In [64]:
torch.sum(torch.nn.functional.one_hot(selected_indices, 233), axis=0)

tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [84]:
mask.shape

torch.Size([223])

In [97]:
def sample_positives_from_mask(boolean_mask, n):
    assert len(boolean_mask.shape) == 1
    positive_indices = torch.masked_select(torch.arange(0, len(boolean_mask)), mask)
    return torch.multinomial(positive_indices.to(torch.float32), n, replacement=False).to(torch.int64)

sample = sample_positives_from_mask(mask, 10)
sample

tensor([149, 138,  79, 135,  99, 108,  67, 127,  64,  46])

In [126]:
def indices_to_mask(indices, depth):
    return torch.sum(torch.nn.functional.one_hot(indices, depth), dim=0).to(torch.bool)

print(indices_to_mask(sample, depth=223))
print(torch.logical_not(indices_to_mask(sample, depth=223)))

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, 

torch.int32

In [33]:
idx2symbol_cell = dict()
with open('../example/experiments.txt') as f:
    for i, line in enumerate(f):
        symbol, cell = line.strip().split('_')
        idx2symbol_cell[i] = (symbol, cell)
torch.save(idx2symbol_cell, 'ENCODE.idx2symbol-cell.pt')

In [30]:
sum(mask_unique_ENCODE_prefer_HepG2)

tensor(150)

In [65]:
example = next(iter(tfrecord_to_dataloader('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord', batch_size=16)))

In [68]:
example[1][:, :, mask_K562].shape

torch.Size([16, 1000, 120])

In [ ]:
torch.logical_and()

In [167]:
class TFIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, filepath, features_filepath=None, batch_size=64, cache=True, shuffle=None):
        super(TFIterableDataset).__init__()
        
        # load tfrecord file and create tf.data pipeline 
        self.dataset = self._load_dataset(filepath, features_filepath, batch_size, cache, shuffle)

    def _load_dataset(self, filepath, features_filepath=None, batch_size=64, cache=True, shuffle=None):
        # no not serialize - only after shuffle/cache 
        dataset = io.dataset_ops.load_tfrecord(filepath, deserialize=False)
        if cache:
            dataset = dataset.cache()
        if shuffle:
            dataset = dataset.shuffle(shuffle)

        # deserialize proto to example
        if features_filepath is None:
            features_filepath = filepath + '.features.json'
        self.features = io.dataset_ops.features_from_json_file(features_filepath)
        dataset = io.dataset_ops.deserialize_dataset(dataset, self.features)

        # batch & prefetch
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)

        # format example & prefetch
        dataset = dataset.map(self._format_example, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
        return dataset
    
    def _format_example(self, example):
        # move channel dim from -1 to -2
        # example['inputs']['input'] = tf.transpose(example['inputs']['input'], perm=[0, 2, 1])
        # example['outputs']['signal']['total'] = tf.transpose(example['outputs']['signal']['total'], perm=[0, 2, 1])

        example = {
            'inputs': {
                'sequence': tf.transpose(example['inputs']['input'], perm=[0, 2, 1])},
            'outputs': {
                'total': tf.transpose(example['outputs']['signal']['total'], perm=[0, 2, 1]),
                'control': tf.transpose(example['outputs']['signal']['control'], perm=[0, 2, 1]),
            },
        }

        # return (input: Tensor, output: Tensor)
        return example
    
    def process_example(self, example):
        return example
    
    def _to_pytorch_compatible(self, example):
        return tf.nest.map_structure(lambda x: torch.tensor(x).to(torch.float32), example)

    def __iter__(self):
        for example in self.dataset.as_numpy_iterator():
            processed_pytorch_example = self._to_pytorch_compatible(self.process_example(example))
            yield processed_pytorch_example['inputs'], processed_pytorch_example['outputs']

In [182]:
class MaskedTFIterableDataset(TFIterableDataset):
    def __init__(self, masks=None, **kwargs):
        super(MaskedTFIterableDataset, self).__init__(**kwargs)
        self.composite_mask = None
        if masks is not None:
            self.composite_mask = self._make_composite_mask(masks)

    def _make_composite_mask(self, masks):
        composite_mask = masks[0]
        for mask in masks[1:]:
            composite_mask = torch.logical_and(composite_mask, mask)
        return composite_mask
    
    def mask_structure(self, structure, mask):
        return tf.nest.map_structure(lambda tensor: tensor[:, :, mask], structure)

    def process_example(self, example):
        if self.composite_mask is not None:
            example['outputs'] = self.mask_structure(example['outputs'], self.composite_mask)
        return example

In [232]:
class MeanESMEmbeddingMaskedTFIterableDataset(MaskedTFIterableDataset):
    def __init__(self, embedding_matrix_filepath, masks=None, **kwargs):
        super(MeanESMEmbeddingMaskedTFIterableDataset, self).__init__(masks, **kwargs)
        self.embedding_matrix = torch.load(embedding_matrix_filepath)
    
    def process_example(self, example):
        # add protein embedding to inputs
        example['inputs']['embedding'] = self.embedding_matrix[self.composite_mask] if self.composite_mask is not None else self.embedding_matrix
        if self.composite_mask is not None:
            example['outputs'] = self.mask_structure(example['outputs'], self.composite_mask)
        return example

In [234]:
mask_HepG2 = torch.load('experiment-mask.HepG2.pt')
# idx2esm = torch.load('../example/esm2_t33_650M_UR50D.ENCODE.idx2mean.pt')
# idx2esm.shape

esm_masked_dataset = MeanESMEmbeddingMaskedTFIterableDataset(embedding_matrix_filepath='../example/esm2_t33_650M_UR50D.ENCODE.idx2mean.pt', masks=[mask_HepG2], filepath='../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord')
# esm_masked_dataset = MeanESMEmbeddingMaskedTFIterableDataset(embedding_matrix_filepath='../example/esm2_t33_650M_UR50D.ENCODE.idx2mean.pt', filepath='../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord')
example = next(iter(esm_masked_dataset))
print(example[0].keys())
print(example[0]['embedding'].shape)
print(example[1]['total'].shape)

/tmp/ipykernel_32914/3937880383.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tf.nest.map_structure(lambda x: torch.tensor(x).to(torch.float32), example)


dict_keys(['sequence', 'embedding'])
torch.Size([103, 1280])
torch.Size([64, 1000, 103])


In [ ]:
mask_HepG2 = torch.load('experiment-mask.HepG2.pt')

masked_dataset = MaskedTFIterableDataset(masks=[mask_HepG2], filepath='../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord')
example = next(iter(masked_dataset))
example[1]['total'].shape

In [177]:
example = next(iter(TFIterableDataset('../example/data.matrix/windows.chr13.4.data.matrix.filtered.tfrecord')))
example[1]['control'].shape

torch.Size([64, 1000, 223])

In [132]:
example[1]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [144]:
# torch.tensor('abc'.encode('UTF-8'), dtype=torch.uint8)

In [145]:
# int.from_bytes('abc'.encode('UTF-8'), byteorder='big')

In [149]:
idx2esm = torch.load('../example/esm2_t33_650M_UR50D.ENCODE.idx2mean.pt')
idx2esm.shape

torch.Size([223, 1280])

In [179]:
idx2esm[mask_HepG2].shape

torch.Size([103, 1280])

In [148]:
idx2esm.shape

torch.Size([223, 1280])

In [19]:
len(torch.load('experiment-mask.HepG2.pt'))

223

In [105]:
y = example[1]
y.shape

torch.Size([64, 1000, 223])

In [123]:
import torchmetrics

class BatchedPCC(torchmetrics.MeanMetric):
    def __init__(self, min_height=2, min_count=None):
        super(BatchedPCC, self).__init__()

        self.min_height = min_height
        self.min_count = min_count

    def update(self, y_pred: torch.Tensor, y: torch.Tensor):
        if y_pred.shape != y.shape:
            raise ValueError('shapes y_pred {y_pred.shape} and y {y.shape} are not the same. ')

        mean_pcc = self._compute_mean_pcc(y_pred, y)

        # update
        super().update(mean_pcc)

    def _compute_mean_pcc(self, y_pred: torch.Tensor, y: torch.Tensor):
        values = []
        for i in range(y.shape[0]):
            values.append(torchmetrics.functional.pearson_corrcoef(y[i], y_pred[i]))
        # stack to (batch_size x ...) - at this point the shape should be (batch_size x experiments
        values = torch.stack(values)

        # create boolean tensor of entries that are *not* NaNs
        values_is_not_nan_mask = torch.logical_not(torch.isnan(values))
        # convert nan's to 0
        values = torch.nan_to_num(values, 0.0)

        # check if required height is reached per experiment
        if self.min_height is not None:
            # should be shape (batch_size, experiments)
            y_min_height_mask = (torch.max(y, dim=-2).values >= self.min_height)
        else:
            y_min_height_mask = torch.ones(*values.shape)
        
        # check if required count is reached per experiment
        if self.min_count is not None:
            # should be shape (batch_size, experiments)
            y_min_count_mask = (torch.sum(y, dim=-2) >= self.min_count)
        else:
            y_min_count_mask = torch.ones(*values.shape)
        
        # boolean mask indicating which experiment (in each batch) passed nan, heigh and count (and is thus used for the final mean PCC)
        passed_boolean_mask = torch.sum(torch.stack([values_is_not_nan_mask, y_min_height_mask, y_min_count_mask]), dim=0) > 0

        # mask out (i.e. zero) all PCC values that did not pass
        values_masked = torch.mul(values, passed_boolean_mask.to(torch.float32))

        # compute mean by only dividing by #-elements that passed
        values_mean = torch.sum(values_masked)/torch.sum(passed_boolean_mask)

        # if ignore_nan:
        #     # only divide by #-elements not NaN
        #     values_mean = torch.sum(values)/torch.sum(values_is_not_nan)
        # else:
        #     values_mean = torch.mean(values)
        
        return values_mean


m = BatchedPCC(min_height=2, min_count=5)
m(torch.rand(*y.shape), y)

tensor(-0.0005)

In [116]:
torch.max(torch.rand(2, 2, 3), dim=-1).values

tensor([[0.8121, 0.5983],
        [0.9303, 0.6061]])

In [109]:
torch.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [201]:
example[0]['embedding'].shape

torch.Size([103, 1280])

In [204]:
torch.nn.Linear(in_features=1280, out_features=256)(example[0]['embedding']).shape

torch.Size([103, 256])

In [229]:
import torch.nn as nn

from torchrbpnet.layers import Conv1DFirstLayer, Conv1DResBlock, LinearProjection

class ProteinEmbeddingMultiRBPNet(nn.Module):
    def __init__(self, n_tasks, n_layers=9, n_body_filters=256):
        super(ProteinEmbeddingMultiRBPNet, self).__init__()

        self.n_tasks = n_tasks

        # layers RNA
        self.body = nn.Sequential(*[Conv1DFirstLayer(4, n_body_filters, 6)]+[(Conv1DResBlock(n_body_filters, n_body_filters, dilation=(2**i))) for i in range(n_layers)])
        self.rna_projection = nn.Linear(in_features=n_body_filters, out_features=256, bias=False)

        # layers protein
        self.protein_projection = nn.Linear(in_features=1280, out_features=256, bias=False)

    def forward(self, inputs, **kwargs):
        # forward RNA
        x_r = inputs['sequence']
        for layer in self.body:
            x_r = layer(x_r)
        # transpose: # (batch_size, dim, N) --> (batch_size, N, dim)
        x_r = torch.transpose(x_r, dim0=-2, dim1=-1)
        # project: (batch_size, N, dim) --> (batch_size, N, new_dim)
        x_r = self.rna_projection(x_r)
        
        # forward protein
        x_p = inputs['embedding']
        x_p = self.protein_projection(x_p)
        # x_r: (#proteins, dim)

        # transpose representations for matmul
        # x_r = torch.transpose(x_r, dim0=-2, dim1=-1) # (batch_size, N, dim)
        x_p = torch.transpose(x_p, dim0=1, dim1=0) # (dim, #proteins)
        
        return torch.matmul(x_r, x_p) # (batch_size, N, #proteins)

network = ProteinEmbeddingMultiRBPNet(n_tasks=223)

In [230]:
y_pred = network(example[0])
y_pred.shape

torch.Size([64, 1000, 103])

In [210]:
# y_pred = network(example[0])
# print(y_pred[0].shape)
# print(y_pred[1].shape)

# y_pred_1_t = torch.transpose(y_pred[1], dim0=1, dim1=0)
# y_pred_1_t.shape

# y_pred_0_t = torch.transpose(y_pred[0], dim0=-2, dim1=0)
# y_pred_0_t.shape

torch.Size([64, 256, 1000])
torch.Size([103, 256])


torch.Size([256, 103])

torch.Size([64, 1000, 256])

In [216]:
# from functorch import vmap

In [215]:
torch.matmul(y_pred_0_t, y_pred_1_t).shape

torch.Size([64, 1000, 103])